In this example we will generate dbc from custom xlsx file with given specification at one excel sheet for multiple messages and signals. For this at first we will produce intermediate data structures to describe messages and corresponding signals, fixing namings. After it - generate dbc one dbc. Some setup are given as input at the beginning, set the manually:
* Matrix_Sheet
* Sender_Name
* IsSigSigned
* IsBig_endian

In [1]:
%load_ext autoreload
%autoreload 2
import dbcgen
import pandas as pd
import re

import cantools
from cantools.database.can import Message, Signal
from cantools.database.conversion import BaseConversion

filein='input/HVH_CAN_MATRIX.xlsx'
IsBig_endian=0
IsSigSigned=0
Sender_Name='ECU'
Matrix_Sheet='Matrix'

ldf is not supported


In [2]:
# read signals sheet
df=pd.read_excel(filein, sheet_name=Matrix_Sheet)
df.head()

,Msg Name\n报文名称,Msg Type\n报文类型,Msg ID (hex)\n报文标识符,Msg Send Type\n报文发送类型,Msg Cycle Time (ms)\n报文周期时间,Msg Length\n(Byte)\n报文长度,Signal Name\n信号名称,Signal Description\n信号描述,Response Error\n状态信息位,Start Byte\n起始字节,...,Signal Max. Value(phys)\n物理最大值,Signal Min. Value (Hex)\n总线最小值,Signal Max. Value(Hex)\n总线最大值,Unit\n单位,Initial Value(Hex)\n初始值,Invalid Value(Hex)无效值,Signal Value Description (hex)\n信号值描述,Remark\n备注,HVAC_HV_PTC_Rq,HV_PTC
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HVAC_HV_PTC_Rq \n(Receive Commands),Normal,0x18AC5A7F,Cycle,1000.0,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tx,Rx
2,NaN,NaN,NaN,NaN,NaN,NaN,HV_PTC_heating_rq,加热器功率要求百分比\nPercentage of Heating Power Request,NaN,0.0,...,100.0,0x0,0x64,%,0x0,0x65~0x7F,0X0~0X64:valid\n0x65~0x7F:invalid,NaN,NaN,r
3,NaN,NaN,NaN,NaN,NaN,NaN,HV_PTC_On_Rq,PTC使能\nEnable HV-PTC,NaN,1.0,...,3.0,0x0,0x3,NaN,0x03,0x03,0x0:PTC_OFF\n0x1:PTC_ON\n0x2:reserved / RES\n0...,NaN,NaN,r
4,NaN,NaN,NaN,NaN,NaN,NaN,HV_PTC_Reset_Rq,复位请求\nReset Request,NaN,1.0,...,3.0,0x0,0x3,NaN,0x03,0x03,0x0:No Reset requested / NO_RESET\n0x1:Reset r...,NaN,NaN,r


In [3]:
# normalize column names

# Helper function to delete all Chinese symbols
def clean_name(name):
    if pd.isna(name):
        return name
    # Remove all Chinese characters
    cleaned = re.sub(r'[\u4e00-\u9fff]', '', str(name))
    # cleaned = re.sub(r'\n', '', cleaned)
    return cleaned.strip()

# Clean the column names - keep only English parts
def clean_column_name(col):
    col1 = clean_name(col)
    col1 = re.sub(r'\s+', '_', col1)  # Replace multiple spaces with single space
    col1 = re.sub(r'\(+', '_', col1)  # 
    col1 = re.sub(r'\)+', '_', col1)  # 
    col1 = re.sub(r'\.+', '', col1)  # 
    col1 = re.sub(r'__+', '_', col1)  # 
    col1= col1.strip('_')  # Remove leading/trailing underscores
    col1= col1.lower()  # Convert to lowercase
    return col1

# Apply cleaning to column names
df.columns = [clean_column_name(col) for col in df.columns]
df.head()

,msg_name,msg_type,msg_id_hex,msg_send_type,msg_cycle_time_ms,msg_length_byte,signal_name,signal_description,response_error,start_byte,...,signal_max_value_phys,signal_min_value_hex,signal_max_value_hex,unit,initial_value_hex,invalid_value_hex,signal_value_description_hex,remark,hvac_hv_ptc_rq,hv_ptc
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HVAC_HV_PTC_Rq \n(Receive Commands),Normal,0x18AC5A7F,Cycle,1000.0,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tx,Rx
2,NaN,NaN,NaN,NaN,NaN,NaN,HV_PTC_heating_rq,加热器功率要求百分比\nPercentage of Heating Power Request,NaN,0.0,...,100.0,0x0,0x64,%,0x0,0x65~0x7F,0X0~0X64:valid\n0x65~0x7F:invalid,NaN,NaN,r
3,NaN,NaN,NaN,NaN,NaN,NaN,HV_PTC_On_Rq,PTC使能\nEnable HV-PTC,NaN,1.0,...,3.0,0x0,0x3,NaN,0x03,0x03,0x0:PTC_OFF\n0x1:PTC_ON\n0x2:reserved / RES\n0...,NaN,NaN,r
4,NaN,NaN,NaN,NaN,NaN,NaN,HV_PTC_Reset_Rq,复位请求\nReset Request,NaN,1.0,...,3.0,0x0,0x3,NaN,0x03,0x03,0x0:No Reset requested / NO_RESET\n0x1:Reset r...,NaN,NaN,r


In [4]:
# separate messages into different DataFrames

def clean_msg_name(col):
    if pd.isna(col):
        return col, ''
    
    name = clean_name(col)
    
    # Extract comment (text within parentheses)
    comment_match = re.search(r'\((.*?)\)', name)
    comment = comment_match.group(1) if comment_match else ''
    
    # Remove comment from name
    if comment_match:
        name = re.sub(r'\(.*?\)', '', name).strip()
    
    return name, comment

msg_name_col = 'msg_name'  # Adjusted to normalized column name
msg_comment_col = 'msg_comment'

msgs_cols = [msg_name_col, msg_comment_col, 'msg_type', 'msg_id_hex', 'msg_send_type', 'msg_cycle_time_ms', 'msg_length_byte' ]
# Fill forward the Msg Name column to group messages properly

# clean the msg_name column first and extract comments
# create a new column for comments
df[msg_comment_col] = None
df[[msg_name_col,msg_comment_col]] = df[msg_name_col].apply(
    lambda x: pd.Series(clean_msg_name(x))
)


df[msg_name_col] = df[msg_name_col].fillna(method='ffill')

# Remove completely empty rows
df = df.dropna(how='all')

# Create separate DataFrames for each message
msgs_df = {}
msgs_info = {}
for msg_name in df[msg_name_col].unique():
    if pd.notna(msg_name) and msg_name.strip():
        msg_df = df[df[msg_name_col] == msg_name].copy()
        # Remove rows where all values are NaN
        msg_df = msg_df.dropna(how='all')
        # extract message infor as first row
        msg_info = msg_df.iloc[0][msgs_cols].to_dict()
        # drop the message info row from the DataFrame
        msg_df = msg_df.iloc[1:].reset_index(drop=True)
        # drop columns related to message info
        msg_df = msg_df.drop(columns=msgs_cols)
        msgs_info[msg_name] = msg_info
        msgs_df[msg_name] = msg_df
# 
#  clean signal descriptions
for msg_name, msg_df in msgs_df.items():
    if 'signal_description' in msg_df.columns:
        msg_df['signal_description'] = msg_df['signal_description'].apply(clean_name)
        
        
# Display the available message DataFrames
print("Available Message DataFrames:")
print(msgs_info)

for msg_name in msgs_df.keys():
    print(f"- {msg_name}")

# Access individual DataFrames like this:
# hvac_ptc_rq_df = message_dfs['HVAC_HV_PTC_Rq']
# ptc_frm1_df = message_dfs['HV_PTC_Frm1']
# etc.

# Show the structure of one DataFrame as example
if msgs_df:
    first_msg = list(msgs_df.keys())[0]
    print(f"\nStructure of '{first_msg}':")
    print(msgs_df[first_msg].head())

Available Message DataFrames:
{'HVAC_HV_PTC_Rq': {'msg_name': 'HVAC_HV_PTC_Rq', 'msg_comment': 'Receive Commands', 'msg_type': 'Normal', 'msg_id_hex': '0x18AC5A7F', 'msg_send_type': 'Cycle', 'msg_cycle_time_ms': 1000.0, 'msg_length_byte': 3.0}, 'HV_PTC_Frm1': {'msg_name': 'HV_PTC_Frm1', 'msg_comment': 'Feedback info', 'msg_type': 'Normal', 'msg_id_hex': '0x18AD7F5A', 'msg_send_type': 'Cycle', 'msg_cycle_time_ms': 1000.0, 'msg_length_byte': 8.0}, 'HVAC_HV_PTC_Rq2': {'msg_name': 'HVAC_HV_PTC_Rq2', 'msg_comment': 'Receive command', 'msg_type': 'Normal', 'msg_id_hex': '0x18ACE48D', 'msg_send_type': nan, 'msg_cycle_time_ms': nan, 'msg_length_byte': 8.0}, 'HV_PTC_Frm2': {'msg_name': 'HV_PTC_Frm2', 'msg_comment': 'Feedback info', 'msg_type': 'Normal', 'msg_id_hex': '0x18AD8DE4', 'msg_send_type': nan, 'msg_cycle_time_ms': nan, 'msg_length_byte': 8.0}}
- HVAC_HV_PTC_Rq
- HV_PTC_Frm1
- HVAC_HV_PTC_Rq2
- HV_PTC_Frm2

Structure of 'HVAC_HV_PTC_Rq':
           signal_name                   signal_d

C:\Users\ivanm\AppData\Local\Temp\ipykernel_13268\2377174454.py:33: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[msg_name_col] = df[msg_name_col].fillna(method='ffill')


In [5]:
#  generate dbc file
def generate_dbc_from_df(msgs_info,msgs_df, output_dbc_file="output.dbc"):

    # Create a new CAN database
    db = cantools.database.can.Database()
    
    # Get unique sender - receivers (nodes)
    # nodes = list(df['Receiver'].unique())
    # for node in nodes:
    #     db.nodes.append(cantools.database.can.Node(name=node))
    
    # Group signals by message
    # messages = df.groupby(['Message_Name', 'Message_ID'])
    
    # Process each message
    for msg_name, msg in msgs_info.items():
        
        signals= msgs_df[msg_name]
        # Create signal list for the message
        signal_list = []
        for _, signal in signals.iterrows():
            # Create signal object
            # check if signal is not NaN
            if pd.isna(signal['signal_name']) or pd.isna(signal['start_bit']):
                continue
            
            conversion = BaseConversion.factory(scale=signal['resolution'], offset=signal['offset'])
            sig = Signal(
                name=signal['signal_name'],
                start=int(signal['start_bit']),
                length=int(signal['bit_length_bit']),
                byte_order='big_endian' if IsBig_endian==1 else 'little_endian',
                is_signed=IsSigSigned==1,
                conversion=conversion,
                # scale=signal['Factor'],
                # offset=signal['Offset'],
                minimum=signal['signal_min_value_phys'] if pd.notna(signal['signal_min_value_phys']) else None,
                maximum=signal['signal_max_value_phys'] if pd.notna(signal['signal_max_value_phys']) else None,
                unit=signal['unit'] if pd.notna(signal['unit']) else None,
                comment=signal['signal_description'] if pd.notna(signal['signal_description']) else '',
                # receivers=[signal['Receiver']]
            )
            signal_list.append(sig)
        
        frame_id=int(msg['msg_id_hex'],16)
        isExtended = frame_id > 0x7FF
        msg = Message(
            frame_id=frame_id,
            name=msg_name,
            length=int(msg['msg_length_byte']),
            signals=signal_list,
            senders=[Sender_Name],  # Assuming receiver is also sender
            cycle_time=msg['msg_cycle_time_ms'] if pd.notna(msg['msg_cycle_time_ms']) else 100,
            is_extended_frame=isExtended,
            comment=msg['msg_comment'] if pd.notna(msg['msg_comment']) else '',
            strict=False
        )
        
        # Add message to database
        db.messages.append(msg)
    
    # Save the database to a DBC file
    # with open(output_dbc_file, 'wb') as f:
    #     cantools.database.dump_file(db, f, 'dbc')
    cantools.database.dump_file(db, output_dbc_file, 'dbc')
    print(f"DBC file created: {output_dbc_file}")

In [6]:
# prepare data for dbcgen
generate_dbc_from_df(msgs_info,msgs_df, output_dbc_file=filein.replace('.xlsx','.dbc'))

DBC file created: input/HVH_CAN_MATRIX.dbc
